In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2
import sys
sys.path.append('../')
from fastai.imports import *
from fastai.dataset import *

In [2]:
m_vgg = to_gpu(vgg16(True)).eval()
set_trainable(m_vgg, False)

In [3]:
def gram(input):
        b,c,h,w = input.size()
        x = input.view(b*c, -1)
        return torch.mm(x, x.t())/input.numel()*1e6

In [4]:
PATH = Path(".")
jeans_bl = PATH/"black-jeans.jpg"
jeans_bl2 = PATH/"black-jeans2.png"
jeans_blue = PATH/"blue-jeans.jpg"
jeans_blue2 = PATH/"blue-jeans2.jpg"
jeans_blue3 = PATH/"blue-jeans3.jpg"
jeans_blue4 = PATH/"blue-jeans4.jpg"
jeans = [jeans_bl2, jeans_blue, jeans_blue2, jeans_blue3, jeans_blue4]
jeans_imgs = [open_image(i) for i in jeans]

In [5]:
sz=288
_, val_tfms = tfms_from_model(vgg16, sz)

In [6]:
block_ends = [i-1 for i,o in enumerate(children(m_vgg)) if isinstance(o,nn.MaxPool2d)]
block_ends

[5, 12, 22, 32, 42]

In [7]:
class SaveFeatures():
    features=None
    def __init__(self, m): self.hook = m.register_forward_hook(self.hook_fn)
    def hook_fn(self, module, input, output): self.features = output
    def close(self): self.hook.remove()

In [8]:
sfs = [SaveFeatures(children(m_vgg)[idx]) for idx in block_ends]

In [12]:
grams = []
for j in jeans_imgs:
    j_tfm = val_tfms(j)
    m_vgg(VV(j_tfm[None]))
    act = sfs[1].features.clone()
    grams.append(to_np(gram(act).view(-1)))
grams = np.asarray(grams)
grams

array([[  4.20599,   4.58014,   0.49505, ...,   9.81263,   0.29686,  28.27554],
       [755.42413, 257.38913,  13.28785, ...,  12.08902,   0.65215,  26.0744 ],
       [805.5143 , 351.8808 ,  57.06281, ...,  20.67003,   1.69265,  51.51708],
       [495.93683, 228.45882,  28.70086, ...,  24.19106,   2.52484,  46.69821],
       [943.58514, 263.47922, 104.6348 , ...,  26.53894,   1.38146,  50.93338]], dtype=float32)

In [13]:
grams.shape

(5, 16384)

In [ ]:
np.save("test.npy", grams)

In [ ]:
# m_vgg(VV(jeans_tfm[None]))
# targ_vs = [V(o.features.clone()) for o in sfs]
# [o.shape for o in targ_vs]

In [14]:
import nmslib

def create_index(a):
    index = nmslib.init(space='angulardist')
    index.addDataPointBatch(a)
    index.createIndex()
    return index

def get_knns(index, vecs):
     return zip(*index.knnQueryBatch(vecs, k=4, num_threads=4))

def get_knn(index, vec): return index.knnQuery(vec, k=4)

In [15]:
index = create_index(grams)

In [16]:
j_tfm = val_tfms(open_image(jeans_bl))
m_vgg(VV(j_tfm[None]))
act = sfs[1].features.clone()

In [17]:
act = to_np(gram(act).view(-1))

In [18]:
act.shape

(16384,)

In [19]:
get_knn(index, act)[0]

array([3, 0, 1, 2], dtype=int32)

In [20]:
jeans[0]

PosixPath('black-jeans2.png')

In [28]:
index.saveIndex("test.bin")

In [30]:
newIndex = nmslib.init(space='angulardist')
newIndex.addDataPointBatch(grams)
newIndex.loadIndex("test.bin")

In [31]:
newIndex.knnQuery(act, k=4)

(array([3, 0, 1, 2], dtype=int32),
 array([0.37905, 0.47742, 0.5659 , 0.5801 ], dtype=float32))